In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col

spark = SparkSession.builder.appName("ETRM Data Filter").getOrCreate()

In [0]:
files =[f.name for f in  dbutils.fs.ls('/Volumes/workspace/default/databrics_data/')]
print(files)

In [0]:
# Loading all the type of files and filter logic
files = ['Cola.xlsx', 'CompanyData.txt', 'etrm_trades.csv', 'iris.json', 'yiedl_daily.parquet']

csv_files = [f for f in files if f.endswith('.csv')]

print("CSV files only:", csv_files)


In [0]:
# Read CSV file with headers and infer schema
etrm_df = spark.read.option("header", True).option("inferSchema", True).csv("/Volumes/workspace/default/databrics_data/etrm_trades.csv")

# Clean column names to remove invalid characters
etrm_df = etrm_df.withColumnRenamed("Trade ID", "Trade_ID") \
                 .withColumnRenamed("Energy Type", "Energy_Type") \
                 .withColumnRenamed("Quantity (MWh)", "Quantity_MWh") \
                 .withColumnRenamed("Total Price (USD)", "Total_Price_USD") \
                 .withColumnRenamed("Price (USD/MWh)", "Price_USD_MWh") \
                 .withColumnRenamed("Date", "Trade_Date")

# Display first few records
display(etrm_df.limit(5))

# Register as Temp View for SQL queries
etrm_df.createOrReplaceTempView("etrm_trades_view")

# Verify registration
display(etrm_df.limit(5))

In [0]:
# Energy Summary by Energy Type
energy_summary = spark.sql("""
    SELECT 
        Energy_Type,
        SUM(Quantity_MWh) AS Total_Quantity_MWh,
        SUM(Total_Price_USD) AS Total_Revenue_USD,
        ROUND(AVG(Price_USD_MWh), 2) AS Avg_Price_USD_MWh
    FROM etrm_trades_view
    GROUP BY Energy_Type
    ORDER BY Total_Revenue_USD DESC
""")

display(energy_summary)

# Daily Trade Summary
daily_summary = spark.sql("""
    SELECT 
        Trade_Date,
        COUNT(DISTINCT Trade_ID) AS Num_Trades,
        SUM(Total_Price_USD) AS Daily_Revenue_USD,
        ROUND(AVG(Price_USD_MWh), 2) AS Avg_Price_USD_MWh
    FROM etrm_trades_view
    GROUP BY Trade_Date
    ORDER BY Trade_Date ASC
""")

display(daily_summary)


In [0]:
etrm_df.write.mode("overwrite").format("delta").saveAsTable("etrm_trades_table")

In [0]:
# Analyzing the schema of the csv file
etrm_df.printSchema()

In [0]:
from pyspark.sql.functions import to_timestamp

etrm_df = etrm_df.withColumn(
    "Trade_Date",
    to_timestamp("Trade_Date", "dd-MM-yyyy HH:mm")
)
display(etrm_df.limit(5))

In [0]:
etrm_df.createOrReplaceTempView("etrm_trades_view")
display(etrm_df.limit(5))

In [0]:
spark.sql("DROP TABLE IF EXISTS workspace.default.energy_summary_table")
spark.sql("DROP TABLE IF EXISTS workspace.default.daily_summary_table")

energy_summary.write.saveAsTable("workspace.default.energy_summary_table")
daily_summary.write.saveAsTable("workspace.default.daily_summary_table")

In [0]:
spark.sql("SHOW TABLES IN workspace.default").show(truncate=False)
spark.sql("SELECT * FROM workspace.default.energy_summary_table LIMIT 5").show()
spark.sql("SELECT * FROM workspace.default.daily_summary_table LIMIT 5").show()

In [0]:
from pyspark.sql.functions import col, round

check_df = etrm_df.withColumn(
    "Calculated_Total", round(col("Quantity_MWh") * col("Price_USD_MWh"), 2)
).withColumn(
    "Difference", col("Total_Price_USD") - col("Calculated_Total")
)

display(check_df.select("Trade_ID", "Energy_Type", "Total_Price_USD", "Calculated_Total", "Difference"))

In [0]:
# Energy type summary
from pyspark.sql.functions import sum as spark_sum

energy_summary = (
    etrm_df.groupBy("Energy_Type")
    .agg(
        spark_sum("Quantity_MWh").alias("Total_Quantity"),
        spark_sum("Total_Price_USD").alias("Total_Revenue")
    )
    .orderBy("Total_Revenue", ascending=False)
)

display(energy_summary)


In [0]:
from pyspark.sql.functions import to_timestamp, to_date, sum as spark_sum

daily_summary = (
    etrm_df.withColumn("Trade_Date", to_date(to_timestamp("Trade_Date", "dd-MM-yyyy HH:mm")))
    .groupBy("Trade_Date")
    .agg(
        spark_sum("Total_Price_USD").alias("Daily_Revenue"),
        spark_sum("Quantity_MWh").alias("Daily_Volume")
    )
    .orderBy("Trade_Date")
)

display(daily_summary)

In [0]:
spark.catalog.listTables()

In [0]:
check_df.show(5)
daily_summary.show(5)